In [ ]:
pip install tensorflow==2.4.1 tensorflow_quantum

Load data (We do 1D TFIM chain since 2D data not in this version of TFQ or nightly)

In [ ]:
import tensorflow_quantum as tfq
import tensorflow as tf
import cirq
import numpy as np
import sympy
nspins = 16
qubits = cirq.GridQubit.rect(nspins, 1)

circuits, labels, _, _ = tfq.datasets.tfi_chain(qubits)

labels = np.array(labels)
print(labels)
labels[labels >= 1] = 1.0
labels = labels * 2 - 1
print(labels)
x_train_tfcirc = tfq.convert_to_tensor(circuits)


Create quantum classifier

In [ ]:
def add_layer_nearest_neighbours(circuit, qubits, gate, prefix):
    for i, q in enumerate(zip(qubits, qubits[1:])):
        symbol = sympy.Symbol(prefix + '-' + str(i))
        circuit.append(gate(*q) ** symbol)


def add_layer_single(circuit, qubits, gate, prefix):
    for i, q in enumerate(qubits):
        symbol = sympy.Symbol(prefix + '-' + str(i))
        circuit.append(gate(q) ** symbol)


def create_quantum_model(N, num_layers):
    qubits = cirq.GridQubit.rect(N, 1)
    circuit = cirq.Circuit()
    for l in range(num_layers):
        add_layer_single(circuit, qubits, cirq.X, f"x_{l}")
        add_layer_nearest_neighbours(circuit, qubits, cirq.XX, f"xx_{l}")
        add_layer_single(circuit, qubits, cirq.Z, f"z_{l}")

    readout = [cirq.Z(q) for q in qubits]
    return circuit, readout

Define Hinge loss

In [ ]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)


Run optimization

In [ ]:
circuit, output = create_quantum_model(nspins, 3)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    tfq.layers.PQC(circuit, output),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.tanh)
])
model.compile(
    loss=tf.keras.losses.Hinge(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[hinge_accuracy])

epochs = 150
batch_size = 32

qnn_history = model.fit(
    x_train_tfcirc, labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=[tf.keras.callbacks.EarlyStopping('loss', patience=5)])

predictions = model.predict(x_train_tfcirc)